In [2]:
import pandas as pd              
import numpy as np 
import tensorflow as tf

data = pd.read_csv("bible_data_set (with count and testament).csv")
data

,citation,book,chapter,verse,text,length (charcters),words,testament
0,Genesis 1:1,Genesis,1,1,In the beginning God created the heaven and th...,56,10,old
1,Genesis 1:2,Genesis,1,2,"And the earth was without form, and void; and ...",144,29,old
2,Genesis 1:3,Genesis,1,3,"And God said, Let there be light: and there wa...",56,11,old
3,Genesis 1:4,Genesis,1,4,"And God saw the light, that it was good: and G...",87,17,old
4,Genesis 1:5,Genesis,1,5,"And God called the light Day, and the darkness...",117,22,old
...,...,...,...,...,...,...,...,...
31097,Revelation 22:17,Revelation,22,17,"And the Spirit and the bride say, Come. And le...",170,33,new
31098,Revelation 22:18,Revelation,22,18,For I testify unto every man that heareth the ...,189,37,new
31099,Revelation 22:19,Revelation,22,19,And if any man shall take away from the words ...,209,44,new
31100,Revelation 22:20,Revelation,22,20,"He which testifieth these things saith, Surely...",97,16,new


this is the data we are going to work on.
we have 8 side features (not used):
###### citation -  which is the book, the verse, and the chapter.
###### book - which is the name of the book, also what we want to predict on given verse.
###### chapter - which is the number of chpter in the book.
###### verse - which is  the number of verse in the chapter.
###### text - is the content of the verse.
###### length - is the number of  characters in the verse.
###### words - is the number of words in the verse.
###### testement - new/old.

and 9412 main features which is the words below.

In [3]:
import nltk
from nltk.stem import PorterStemmer

ps = PorterStemmer() 

vocabulary_size = 0
word2location = {}

def prepare_vocabulary(data):
    index = 0
    for sentance in data['text']:
        #sentance = sentance.lower()
        words = nltk.word_tokenize(sentance)
        for word in words:
            stemed_word = ps.stem(word)
            if stemed_word not in word2location:
                word2location[stemed_word] = index
                index += 1
    return index

def convert2vec(sentance):
    #sentance = sentance.lower()
    res_vec = np.zeros(vocabulary_size)
    words = nltk.word_tokenize(sentance)
    for word in words:
        stemed_word = ps.stem(word)
        if stemed_word in word2location:
            res_vec[word2location[stemed_word]]+=1
    return res_vec

vocabulary_size = prepare_vocabulary(data)
print("the size of the vocabulary is: ", vocabulary_size)
word2location

the size of the vocabulary is:  9421


{'In': 0,
 'the': 1,
 'begin': 2,
 'god': 3,
 'creat': 4,
 'heaven': 5,
 'and': 6,
 'earth': 7,
 '.': 8,
 'wa': 9,
 'without': 10,
 'form': 11,
 ',': 12,
 'void': 13,
 ';': 14,
 'dark': 15,
 'upon': 16,
 'face': 17,
 'of': 18,
 'deep': 19,
 'spirit': 20,
 'move': 21,
 'water': 22,
 'said': 23,
 'let': 24,
 'there': 25,
 'be': 26,
 'light': 27,
 ':': 28,
 'saw': 29,
 'that': 30,
 'it': 31,
 'good': 32,
 'divid': 33,
 'from': 34,
 'call': 35,
 'day': 36,
 'he': 37,
 'night': 38,
 'even': 39,
 'morn': 40,
 'were': 41,
 'first': 42,
 'a': 43,
 'firmament': 44,
 'in': 45,
 'midst': 46,
 'made': 47,
 'which': 48,
 'under': 49,
 'abov': 50,
 'so': 51,
 'second': 52,
 'gather': 53,
 'togeth': 54,
 'unto': 55,
 'one': 56,
 'place': 57,
 'dri': 58,
 'land': 59,
 'appear': 60,
 'sea': 61,
 'bring': 62,
 'forth': 63,
 'grass': 64,
 'herb': 65,
 'yield': 66,
 'seed': 67,
 'fruit': 68,
 'tree': 69,
 'after': 70,
 'hi': 71,
 'kind': 72,
 'whose': 73,
 'is': 74,
 'itself': 75,
 'brought': 76,
 'third'

In [19]:
tf.compat.v1.disable_v2_behavior()

books = ['Genesis', 'Exodus', 'Leviticus', 'Numbers', 'Deuteronomy', 'Joshua', 'Judges',
            'Ruth', '1 Samuel', '2 Samuel', '1 Kings', '2 Kings', '1 Chronicles', '2 Chronicles',
            'Ezra', 'Nehemiah', 'Esther', 'Job', 'Psalms', 'Proverbs', 'Ecclesiastes',
            'Song of Solomon', 'Isaiah', 'Jeremiah', 'Lamentations', 'Ezekiel', 'Daniel',
            'Hosea', 'Joel', 'Amos', 'Obadiah', 'Jonah', 'Micah', 'Nahum', 'Habakkuk',
            'Zephaniah', 'Haggai',    'Zechariah',    'Malachi', 'Matthew', 'Mark', 'Luke', 'John', 'Acts', 'Romans', '1 Corinthians',
            '2 Corinthians', 'Galatians', 'Ephesians', 'Philippians', 'Colossians',
            '1 Thessalonians', '2 Thessalonians', '1 Timothy', '2 Timothy', 'Titus', 'Philemon',
            'Hebrews', 'James', '1 Peter', '2 Peter', '1 John', '2 John', '3 John', 'Jude',
            'Revelation']

def encode(line):
    res_vec = np.zeros(66)
    idx = books.index(data.iloc[line]['book'])
    res_vec[idx] = 1
    return res_vec

def logistic_fun(z):
    return 1/(1.0 + np.exp(-z))

features = vocabulary_size
categories = 66 #number of books
epsilon = 1e-12
x = tf.compat.v1.placeholder(tf.float32,[None,features])
y_ = tf.compat.v1.placeholder(tf.float32,[None,categories])
W = tf.compat.v1.Variable(tf.zeros([features,categories]))
b = tf.compat.v1.Variable(tf.zeros([categories]))

y = tf.nn.softmax(tf.matmul(x,W)+b)
loss = -tf.reduce_mean(y_*tf.compat.v1.log(y))

update = tf.compat.v1.train.GradientDescentOptimizer(0.00001).minimize(loss) #0.001

data_x = np.array([convert2vec(data.iloc[i]['text']) for i in range(len(data['text']))])
data_y = np.array([encode(i) for i in range(len(data['text']))])

sess = tf.compat.v1.Session()

sess.run(tf.compat.v1.global_variables_initializer())

display_step = 2
#cost = tf.reduce_sum(tf.pow(y_ - y, 2))/(2*categories)

for i in range(100):
    sess.run(update, feed_dict = {x:data_x, y_:data_y})
    if i % 1 == 0 :
        print('Iteration:' , i , ' W:' , sess.run(W) , ' b:' , sess.run(b), ' loss:', loss.eval(session=sess, feed_dict = {x:data_x, y_:data_y}))
    
# for i in range(len(data['text'])):
#     test = data.iloc[i]['text']
#     t = np.array([convert2vec(test)])
#     print(sess.run(y,feed_dict={x:t}))
    
#y_values = tf.add(tf.matmul(x, W), b)
#y = tf.nn.softmax(y_values) 

test1 = "In the beginning God created the heaven and the earth. "
t1 = np.array([convert2vec(test1)])
print("prediction for 1,1: " ,sess.run(y,feed_dict={x:t1})

Iteration: 0  W: [[ 3.3658047e-09  9.3002178e-10  1.4171773e-09 ... -2.4800677e-09
  -2.4800677e-09 -1.9929118e-09]
 [ 7.2559646e-07  1.0446834e-06  7.1439206e-07 ... -4.6403829e-07
  -4.5234651e-07  4.1142254e-08]
 [ 1.9117139e-09 -5.2406146e-10 -1.0112179e-09 ... -1.0112179e-09
  -1.0112179e-09  1.4245608e-09]
 ...
 [-7.3811478e-12 -7.3811478e-12 -7.3811478e-12 ... -7.3811478e-12
  -7.3811478e-12  4.7977455e-10]
 [-7.3811478e-12 -7.3811478e-12 -7.3811478e-12 ... -7.3811478e-12
  -7.3811478e-12  4.7977455e-10]
 [-7.3811478e-12 -7.3811478e-12 -7.3811478e-12 ... -7.3811478e-12
  -7.3811478e-12  4.7977455e-10]]  b: [ 5.16897160e-07  3.61030175e-07  1.88699858e-07  3.97589133e-07
  2.37344196e-07  9.10053686e-08  7.15640809e-08 -1.88106910e-07
  1.64924018e-07  1.09018664e-07  1.67859255e-07  1.20708734e-07
  2.29143993e-07  1.70812470e-07 -9.31438606e-08 -3.17556648e-08
 -1.48163565e-07  2.91510162e-07  9.69945745e-07  2.16105619e-07
 -1.21366270e-07 -1.72497622e-07  3.99731249e-07  4.34

Iteration: 5  W: [[ 2.0194463e-08  5.5798379e-09  8.5029015e-09 ... -1.4880234e-08
  -1.4880240e-08 -1.1957476e-08]
 [ 4.3534928e-06  6.2680238e-06  4.2863107e-06 ... -2.7841893e-06
  -2.7140400e-06  2.4685124e-07]
 [ 1.1470140e-08 -3.1445064e-09 -6.0673822e-09 ... -6.0672316e-09
  -6.0672347e-09  8.5473619e-09]
 ...
 [-4.4288968e-11 -4.4288760e-11 -4.4288004e-11 ... -4.4285898e-11
  -4.4285933e-11  2.8786471e-09]
 [-4.4288968e-11 -4.4288760e-11 -4.4288004e-11 ... -4.4285898e-11
  -4.4285933e-11  2.8786471e-09]
 [-4.4287886e-11 -4.4287778e-11 -4.4287372e-11 ... -4.4286439e-11
  -4.4286450e-11  2.8786471e-09]]  b: [ 3.1013831e-06  2.1661797e-06  1.1321985e-06  2.3855337e-06
  1.4240642e-06  5.4603140e-07  4.2938407e-07 -1.1286402e-06
  9.8954320e-07  6.5411132e-07  1.0071540e-06  7.2424928e-07
  1.3748581e-06  1.0248638e-06 -5.5886278e-07 -1.9053402e-07
 -8.8898105e-07  1.7490607e-06  5.8196347e-06  1.2966337e-06
 -7.2819716e-07 -1.0349848e-06  2.3983455e-06  2.6088780e-06
 -9.2683661e-

Iteration: 10  W: [[ 3.70225415e-08  1.02291464e-08  1.55883377e-08 ... -2.72801408e-08
  -2.72801586e-08 -2.19220571e-08]
 [ 7.98124620e-06  1.14912373e-05  7.85815791e-06 ... -5.10427617e-06
  -4.97567180e-06  4.52554616e-07]
 [ 2.10283027e-08 -5.76518255e-09 -1.11236762e-08 ... -1.11231255e-08
  -1.11231353e-08  1.56701532e-08]
 ...
 [-8.12002618e-11 -8.11995124e-11 -8.11967230e-11 ... -8.11890000e-11
  -8.11891249e-11  5.27751931e-09]
 [-8.12002618e-11 -8.11995124e-11 -8.11967230e-11 ... -8.11890000e-11
  -8.11891249e-11  5.27751931e-09]
 [-8.11963066e-11 -8.11959111e-11 -8.11944192e-11 ... -8.11909984e-11
  -8.11910469e-11  5.27751931e-09]]  b: [ 5.6858689e-06  3.9713273e-06  2.0756959e-06  4.3734753e-06
  2.6107830e-06  1.0010465e-06  7.8719813e-07 -2.0691687e-06
  1.8141614e-06  1.1991998e-06  1.8464401e-06  1.3277636e-06
  2.5205350e-06  1.8788576e-06 -1.0245808e-06 -3.4931244e-07
 -1.6297963e-06  3.2066112e-06  1.0669200e-05  2.3771620e-06
 -1.3350256e-06 -1.8974689e-06  4.396

Iteration: 15  W: [[ 5.3850041e-08  1.4877944e-08  2.2673477e-08 ... -3.9679787e-08
  -3.9679822e-08 -3.1886653e-08]
 [ 1.1608856e-05  1.6714324e-05  1.1429931e-05 ... -7.4242985e-06
  -7.2372404e-06  6.5825196e-07]
 [ 3.0586197e-08 -8.3860892e-09 -1.6180103e-08 ... -1.6178900e-08
  -1.6178920e-08  2.2792937e-08]
 ...
 [-1.1811505e-10 -1.1811339e-10 -1.1810733e-10 ... -1.1809047e-10
  -1.1809075e-10  7.6763920e-09]
 [-1.1811505e-10 -1.1811339e-10 -1.1810733e-10 ... -1.1809047e-10
  -1.1809075e-10  7.6763920e-09]
 [-1.1810640e-10 -1.1810555e-10 -1.1810229e-10 ... -1.1809483e-10
  -1.1809492e-10  7.6763920e-09]]  b: [ 8.2703536e-06  5.7764719e-06  3.0191920e-06  6.3614107e-06
  3.7974994e-06  1.4560280e-06  1.1449794e-06 -3.0096917e-06
  2.6387779e-06  1.7442749e-06  2.6857194e-06  1.9312595e-06
  3.6661884e-06  2.7328206e-06 -1.4902967e-06 -5.0809120e-07
 -2.3706091e-06  4.6641608e-06  1.5518715e-05  3.4576906e-06
 -1.9418512e-06 -2.7599499e-06  6.3951952e-06  6.9565913e-06
 -2.4715550e

Iteration: 20  W: [[ 7.0676947e-08  1.9526235e-08  2.9758326e-08 ... -5.2079169e-08
  -5.2079237e-08 -4.1851269e-08]
 [ 1.5236320e-05  2.1937287e-05  1.5001631e-05 ... -9.7442562e-06
  -9.4987481e-06  8.6394323e-07]
 [ 4.0143824e-08 -1.1007225e-08 -2.1236662e-08 ... -2.1234557e-08
  -2.1234587e-08  2.9915711e-08]
 ...
 [-1.5503329e-10 -1.5503039e-10 -1.5501980e-10 ... -1.5499028e-10
  -1.5499077e-10  1.0075264e-08]
 [-1.5503329e-10 -1.5503039e-10 -1.5501980e-10 ... -1.5499028e-10
  -1.5499077e-10  1.0075264e-08]
 [-1.5501818e-10 -1.5501668e-10 -1.5501099e-10 ... -1.5499793e-10
  -1.5499810e-10  1.0075264e-08]]  b: [ 1.08548375e-05  7.58161332e-06  3.96268615e-06  8.34933689e-06
  4.98421196e-06  1.91097456e-06  1.50270682e-06 -3.95020515e-06
  3.46339129e-06  2.28933436e-06  3.52499319e-06  2.53474582e-06
  4.81183088e-06  3.58676925e-06 -1.95600956e-06 -6.66870960e-07
 -3.11141957e-06  6.12170516e-06  2.03682139e-05  4.53821849e-06
 -2.54867359e-06 -3.62242599e-06  8.39353561e-06  9.1

...

Iteration: 75  W: [[ 2.55734506e-07  7.06237842e-08  1.07672356e-07 ... -1.88455147e-07
  -1.88456013e-07 -1.51463183e-07]
 [ 5.51289049e-05  7.93814543e-05  5.42855778e-05 ... -3.52595707e-05
  -3.43712163e-05  3.12618249e-06]
 [ 1.45260131e-07 -3.98549460e-08 -7.68674226e-08 ... -7.68388944e-08
  -7.68393065e-08  1.08265695e-07]
 ...
 [-5.61363345e-10 -5.61324376e-10 -5.61180435e-10 ... -5.60779922e-10
  -5.60786473e-10  3.64628328e-08]
 [-5.61363345e-10 -5.61324376e-10 -5.61180435e-10 ... -5.60779922e-10
  -5.60786473e-10  3.64628328e-08]
 [-5.61158342e-10 -5.61138025e-10 -5.61060809e-10 ... -5.60883728e-10
  -5.60885949e-10  3.64628328e-08]]  b: [ 3.92840921e-05  2.74379508e-05  1.43408806e-05  3.02160533e-05
  1.80375973e-05  6.91435980e-06  5.43617671e-06 -1.42950457e-05
  1.25334736e-05  8.28438897e-06  1.27561461e-05  9.17206671e-06
  1.74131455e-05  1.29789296e-05 -7.07865092e-06 -2.41388693e-06
 -1.12599992e-05  2.21534738e-05  7.37113587e-05  1.64237626e-05
 -9.22312302e-06 

Iteration: 80  W: [[ 2.7255442e-07  7.5265959e-08  1.1475369e-07 ... -2.0085139e-07
  -2.0085238e-07 -1.6142799e-07]
 [ 5.8754635e-05  8.4602885e-05  5.7856414e-05 ... -3.7578757e-05
  -3.6631976e-05  3.3318076e-06]
 [ 1.5481456e-07 -4.2478849e-08 -8.1925549e-08 ... -8.1893113e-08
  -8.1893589e-08  1.1538836e-07]
 ...
 [-5.9832328e-10 -5.9827904e-10 -5.9811534e-10 ... -5.9766003e-10
  -5.9766747e-10  3.8861696e-08]
 [-5.9832328e-10 -5.9827904e-10 -5.9811534e-10 ... -5.9766003e-10
  -5.9766747e-10  3.8861696e-08]
 [-5.9809024e-10 -5.9806710e-10 -5.9797933e-10 ... -5.9777805e-10
  -5.9778055e-10  3.8861696e-08]]  b: [ 4.1868549e-05  2.9243052e-05  1.5284315e-05  3.2203890e-05
  1.9224221e-05  7.3691217e-06  5.7936886e-06 -1.5235415e-05
  1.3357915e-05  8.8293364e-06  1.3595212e-05  9.7752918e-06
  1.8558641e-05  1.3832585e-05 -7.5443245e-06 -2.5727554e-06
 -1.2000730e-05  2.3610852e-05  7.8560559e-05  1.7504215e-05
 -9.8298233e-06 -1.3971551e-05  3.2371867e-05  3.5212826e-05
 -1.2511594e

Iteration: 85  W: [[ 2.8937376e-07  7.9907622e-08  1.2183473e-07 ... -2.1324738e-07
  -2.1324848e-07 -1.7139286e-07]
 [ 6.2380226e-05  8.9824192e-05  6.1427178e-05 ... -3.9897885e-05
  -3.8892675e-05  3.5374267e-06]
 [ 1.6436871e-07 -4.5102983e-08 -8.6983810e-08 ... -8.6947217e-08
  -8.6947750e-08  1.2251104e-07]
 ...
 [-6.3528671e-10 -6.3523681e-10 -6.3505207e-10 ... -6.3453853e-10
  -6.3454686e-10  4.1260559e-08]
 [-6.3528671e-10 -6.3523681e-10 -6.3505207e-10 ... -6.3453853e-10
  -6.3454686e-10  4.1260559e-08]
 [-6.3502381e-10 -6.3499778e-10 -6.3489869e-10 ... -6.3467165e-10
  -6.3467442e-10  4.1260559e-08]]  b: [ 4.44530051e-05  3.10481482e-05  1.62277374e-05  3.41917184e-05
  2.04108346e-05  7.82386087e-06  6.15118597e-06 -1.61757744e-05
  1.41823275e-05  9.37427103e-06  1.44342484e-05  1.03784778e-05
  1.97041227e-05  1.46862021e-05 -8.00999442e-06 -2.73163118e-06
 -1.27414514e-05  2.50682260e-05  8.34097082e-05  1.85846584e-05
 -1.04365135e-05 -1.48339295e-05  3.43699066e-05  3.7

Iteration: 90  W: [[ 3.0619250e-07  8.4548780e-08  1.2891549e-07 ... -2.2564311e-07
  -2.2564434e-07 -1.8135772e-07]
 [ 6.6005668e-05  9.5045369e-05  6.4997868e-05 ... -4.2216943e-05
  -4.1153307e-05  3.7430400e-06]
 [ 1.7392261e-07 -4.7727347e-08 -9.2042193e-08 ... -9.2001201e-08
  -9.2001798e-08  1.2963369e-07]
 ...
 [-6.7225359e-10 -6.7219769e-10 -6.7199074e-10 ... -6.7141537e-10
  -6.7142464e-10  4.3659423e-08]
 [-6.7225359e-10 -6.7219769e-10 -6.7199074e-10 ... -6.7141537e-10
  -6.7142464e-10  4.3659423e-08]
 [-6.7195904e-10 -6.7192990e-10 -6.7181888e-10 ... -6.7156453e-10
  -6.7156763e-10  4.3659423e-08]]  b: [ 4.70374616e-05  3.28532406e-05  1.71711454e-05  3.61795464e-05
  2.15974396e-05  8.27857639e-06  6.50866696e-06 -1.71161228e-05
  1.50067126e-05  9.91919023e-06  1.52732537e-05  1.09816237e-05
  2.08495894e-05  1.55397775e-05 -8.47565934e-06 -2.89051445e-06
 -1.34821639e-05  2.65256003e-05  8.82587919e-05  1.96650926e-05
 -1.10431947e-05 -1.56963015e-05  3.63679137e-05  3.9

Iteration: 95  W: [[ 3.2301065e-07  8.9189427e-08  1.3599595e-07 ... -2.3803857e-07
  -2.3803995e-07 -1.9132258e-07]
 [ 6.9630965e-05  1.0026643e-04  6.8568486e-05 ... -4.4535944e-05
  -4.3413882e-05  3.9486476e-06]
 [ 1.8347625e-07 -5.0351943e-08 -9.7100710e-08 ... -9.7055064e-08
  -9.7055732e-08  1.3675631e-07]
 ...
 [-7.0922407e-10 -7.0916173e-10 -7.0893130e-10 ... -7.0829054e-10
  -7.0830086e-10  4.6058286e-08]
 [-7.0922407e-10 -7.0916173e-10 -7.0893130e-10 ... -7.0829054e-10
  -7.0830086e-10  4.6058286e-08]
 [-7.0889600e-10 -7.0886347e-10 -7.0873990e-10 ... -7.0845663e-10
  -7.0846012e-10  4.6058286e-08]]  b: [ 4.9621918e-05  3.4658333e-05  1.8114542e-05  3.8167356e-05
  2.2784036e-05  8.7332692e-06  6.8661311e-06 -1.8056462e-05
  1.5831067e-05  1.0464093e-05  1.6112226e-05  1.1584731e-05
  2.1995040e-05  1.6393315e-05 -8.9413197e-06 -3.0494048e-06
 -1.4222868e-05  2.7982975e-05  9.3107810e-05  2.0745520e-05
 -1.1649867e-05 -1.6558666e-05  3.8365899e-05  4.1732510e-05
 -1.4828355e

we now encoded every book to one hot vector.
also we have the bag of words.

right now we see that our model is weak almost all numbers are equal, so we need to improve our model,
but after alot of epochs, the correct prediction get a little more points than other. 

In [9]:
data_x

array([[1., 3., 1., ..., 0., 0., 0.],
       [0., 6., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 5., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [29]:
data_y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [14]:
t1

array([[1., 3., 1., ..., 0., 0., 0.]])